# Sales Forecasting

Walmart has been having trouble figuring out how much to buy to restock its products and optimize sales. On the last important dates for the retail sector, the team had problems with purchasing supplies. In some departments, the purchase was above the expected sales and in others there was a lack of products.

The supply team asked us to help them solve their problem in forecasting demand for each store and each department. The team opted to build a regression model to predict sales and thus help the supply team scale purchases.

# 1.0 Data Catalog

**Dataset - Store**<br>
**Store** - Store IDs <br>
**Type** - Store types <br>
**Size** - Store sizes <br> <br>

**Dataset - Features**<br>
**Store** - Store IDs <br>
**Date** - The week <br>
**Temperature** - Average temperature in the region <br>
**Fuel Price** - Cost of fuel in region <br>
**MarkDown 1-5** - Anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA <br>
**CPI** - The customer price index <br>
**Unemployment** - The unemployment rate <br>
**IsHoliday** - Whether the week is a especial holiday week <br>
 <br>

**Dataset - Train/Test**<br>
**Store** - Store IDs <br>
**Dept** - Departament <br>
**Weekly_sales** - Weekly Sales <br>
**Date** - The first date of the week <br>
**IsHoliday** - True or False <br><br>


For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data):

**Super Bowl:**  <br>
    - 12-Feb-10 <br>
    - 11-Feb-11 <br>
    - 10-Feb-12 <br>
    - 8-Feb-13 <br>
**Labor Day:**  <br>
    - 10-Sep-10 <br>
    - 9-Sep-11 <br>
    - 7-Sep-12 <br>
    - 6-Sep-13 <br>
**Thanksgiving:**  <br>
    - 26-Nov-10 <br>
    - 25-Nov-11 <br>
    - 23-Nov-12 <br>
    - 29-Nov-13 <br>
**Christmas:**  <br>
    - 31-Dec-10 <br>
    - 30-Dec-11 <br>
    - 28-Dec-12 <br>
    - 27-Dec-13 <br>

# 2.0 Libraries

In [59]:
import pandas as pd
import pandasql as ps
import numpy as np

# 3.0 Loading Data

In [10]:
df_stores = pd.read_csv('stores.csv')
df_stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [67]:
df_features = pd.read_csv('features.csv').drop(columns = 'IsHoliday')
df_features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106


In [66]:
df_train_raw = pd.read_csv('train.csv')
df_train_raw.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,is_train
0,1,1,2010-02-05,24924.50,False,1
1,1,1,2010-02-12,46039.49,True,1
2,1,1,2010-02-19,41595.55,False,1
3,1,1,2010-02-26,19403.54,False,1
4,1,1,2010-03-05,21827.90,False,1


In [72]:
df_test_raw = pd.read_csv('test.csv')
df_test_raw.head()

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


## 3.1 Join

In [88]:
query = """
    SELECT df.Store,
           df.Dept,
           df.Date,
           df.Weekly_Sales,
           df.IsHoliday,
           stores.Type,
           stores.Size,
           features.Temperature,
           features.Fuel_Price,
           features.MarkDown1,
           features.MarkDown2,
           features.MarkDown3,
           features.MarkDown4,
           features.MarkDown5,
           features.CPI,
           features.Unemployment
    FROM df_train_raw df
    LEFT JOIN df_stores stores
    ON stores.Store = df.Store 
    LEFT JOIN df_features features
    ON features.Store = df.Store AND features.Date = df.Date;
"""
df_train = ps.sqldf(query)

In [83]:
query = """
    SELECT df.Store,
           df.Dept,
           df.Date,
           df.IsHoliday,
           stores.Type,
           stores.Size,
           features.Temperature,
           features.Fuel_Price,
           features.MarkDown1,
           features.MarkDown2,
           features.MarkDown3,
           features.MarkDown4,
           features.MarkDown5,
           features.CPI,
           features.Unemployment
    FROM df_test_raw df
    LEFT JOIN df_stores stores
    ON stores.Store = df.Store 
    LEFT JOIN df_features features
    ON features.Store = df.Store AND features.Date = df.Date;
"""
df_test = ps.sqldf(query, locals())

# 4.0 Descriptive Analisys

In [89]:
df_train_desc = df_train.copy()
df_test_desc = df_test.copy()

We have missing value in MarkDown features

In [90]:
df_train_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  int64  
 5   Type          421570 non-null  object 
 6   Size          421570 non-null  int64  
 7   Temperature   421570 non-null  float64
 8   Fuel_Price    421570 non-null  float64
 9   MarkDown1     150681 non-null  float64
 10  MarkDown2     111248 non-null  float64
 11  MarkDown3     137091 non-null  float64
 12  MarkDown4     134967 non-null  float64
 13  MarkDown5     151432 non-null  float64
 14  CPI           421570 non-null  float64
 15  Unemployment  421570 non-null  float64
dtypes: float64(10), int64(4), object(2)
memory usage: 51.5+ MB


In [87]:
df_test_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         115064 non-null  int64  
 1   Dept          115064 non-null  int64  
 2   Date          115064 non-null  object 
 3   IsHoliday     115064 non-null  int64  
 4   Type          115064 non-null  object 
 5   Size          115064 non-null  int64  
 6   Temperature   115064 non-null  float64
 7   Fuel_Price    115064 non-null  float64
 8   MarkDown1     114915 non-null  float64
 9   MarkDown2     86437 non-null   float64
 10  MarkDown3     105235 non-null  float64
 11  MarkDown4     102176 non-null  float64
 12  MarkDown5     115064 non-null  float64
 13  CPI           76902 non-null   float64
 14  Unemployment  76902 non-null   float64
dtypes: float64(9), int64(4), object(2)
memory usage: 13.2+ MB


In [93]:
df_test_desc[df_test_desc['CPI'].isna()]

,Store,Dept,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
26,1,1,2013-05-03,0,A,151315,66.66,3.386,2298.63,2.00,129.90,55.46,1301.04,NaN,NaN
27,1,1,2013-05-10,0,A,151315,63.90,3.392,4624.61,83.88,42.38,1618.31,8144.90,NaN,NaN
28,1,1,2013-05-17,0,A,151315,69.53,3.454,16170.50,92.00,8.32,4127.24,6206.97,NaN,NaN
29,1,1,2013-05-24,0,A,151315,77.19,3.494,7959.89,178.00,1621.47,3152.57,2938.70,NaN,NaN
30,1,1,2013-05-31,0,A,151315,78.02,3.466,2369.77,142.45,475.35,45.55,2056.84,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115059,45,98,2013-06-28,0,B,118221,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN
115060,45,98,2013-07-05,0,B,118221,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN
115061,45,98,2013-07-12,0,B,118221,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN
115062,45,98,2013-07-19,0,B,118221,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN


In [99]:
df_features[df_features['Store'] == 1]

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106
...,...,...,...,...,...,...,...,...,...,...,...
177,1,2013-06-28,83.94,3.495,4205.98,796.70,6.84,3816.78,4812.74,NaN,NaN
178,1,2013-07-05,79.85,3.422,7649.99,3503.29,1766.77,9454.96,1079.89,NaN,NaN
179,1,2013-07-12,83.12,3.400,6089.94,1362.42,209.62,2367.42,2651.05,NaN,NaN
180,1,2013-07-19,79.26,3.556,3117.04,1060.39,199.05,1012.30,5381.72,NaN,NaN


# WIP